# REACT

In [2]:
!pip install -q streamlit langchain langchain_experimental langchain-openai langchain-community langchain-text-splitters chromadb python-dotenv pyngrok pypdf

In [ ]:
# 🔑 현아의 ngrok 인증 토큰을 여기에 붙여넣고 실행해주세요!
# "YOUR_AUTHTOKEN_HERE" 부분을 현아의 실제 ngrok 인증 토큰으로 바꿔주세요!
!ngrok config add-authtoken "실제 ngrok 인증토큰 붙여넣기"
print("✅ ngrok 인증 토큰 등록 완료!")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ ngrok 인증 토큰 등록 완료!


In [ ]:
import os

# 직접 환경변수 등록 (가장 확실한 방법)
os.environ["OPENAI_API_KEY"] = "실제 OpenAI API 키 붙여넣기"

# 확인
print("✅ OPENAI_API_KEY 등록 완료:", os.environ.get("OPENAI_API_KEY")[:10] + "...")

✅ OPENAI_API_KEY 등록 완료: sk-proj-eI...


In [5]:
%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader, PyPDFLoader # PDF 로더 추가
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# 기본 메인 화면 설정 페이지 타이틀 / 화면 타이틀
st.set_page_config(page_title = "RealView 챗봇 (AI 기반 지역 분석)")
st.title("🏡 RealView 챗봇: AI 기반 지역 분석 챗봇")
st.markdown("현아가 만든 기획서를 바탕으로 한 'AI 기반 지역 시세 및 분위기 분석 챗봇 RealView'입니다. 궁금한 지역에 대해 편하게 물어봐주세요! ✨")

# 세션 초기화
if "messages" not in st.session_state:
    st.session_state["messages"] = []

# LLM 구성
llm = ChatOpenAI(model = 'gpt-4o-mini')

# RealView의 목적에 맞게 프롬프트를 변경했어요.
prompt = ChatPromptTemplate.from_template("""
너는 'RealView'라는 이름의 친절한 AI 부동산 비서야.
사용자가 특정 지역의 부동산 시세, 거래 동향, 생활 분위기, 학군, 편의시설 등 다양한 정보를 문의하면,
아래 참고 문서를 기반으로 분석해서 가장 현명하고 만족스러운 부동산 의사결정을 돕는 인사이트를 제공해 줘.

질문의 의도를 정확히 파악하고, 필요한 경우 긍정/부정 분위기 분석 등 정성적인 정보도 함께 알려주는 게 중요해.
친근하고 전문적인 어조로 답변해 줘!

참고 문서: {context}
사용자 질문: {q}
답변:
""")
chain = prompt | llm | StrOutputParser()

os.makedirs("data", exist_ok=True)
realview_doc_path = "data/realview_data.md"

if not os.path.exists(realview_doc_path):
    with open(realview_doc_path, "w", encoding="utf-8") as f:
        f.write(
            "## 강남구 역삼동 아파트 시세 분석\n"
            "- '개나리 푸르지오' 아파트 전용 84㎡ 기준 최근 실거래가 23억원 (지난달 대비 5% 상승, 전년 대비 15% 상승).\n"
            "- 전세가율 65%, 매매/전세 동반 상승세.\n"
            "- 주변 '역삼초등학교' 학군 우수 평가.\n\n"
            "## 역삼동 지역 커뮤니티 분위기\n"
            "- 최근 '역삼동맘카페' 게시판: 'OO아파트 재건축 논의' 관련 긍정적인 의견 다수. ('대형 카페 입점 소식'으로 지역 활기 띤다).\n"
            "- '직장인 익명 게시판': 밤늦게까지 영업하는 맛집 많아 만족도 높음. (다만 주차 문제가 심각하다는 불평도 존재).\n\n"
            "## 강남구 청담동 지역 정보\n"
            "- '청담 브릿지' 상가 공실률 감소, 명품 편집샵 입점 증가로 상권 활기.\n"
            "- '청담공원' 리모델링 예정으로 녹지 공간 개선 기대.\n"
            "- '청담고등학교' 명문 학군 유지 중.\n\n"
            "## 최신 부동산 뉴스\n"
            "- 2025년 11월 7일: 정부, 강남 3구 부동산 규제 완화 시사. 기대감에 매수세 유입 가능성.\n"
            "- 2025년 11월 9일: 대형 건설사, 역삼동 고급 주상복합 착공 소식 발표."
        )

loader = TextLoader(realview_doc_path, encoding = 'utf-8')
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 150)
chunks = splitter.split_documents(docs)

emb = OpenAIEmbeddings(model = 'text-embedding-3-small')
db  = Chroma.from_documents(chunks, emb)

# Streamlit UI
st.subheader("궁금한 지역 정보를 물어보세요!")

q = st.chat_input("예: 역삼동 아파트 시세는 어때요? 또는 청담동 분위기는 어떤가요?")

if q:
    with st.chat_message("user"):
        st.markdown(q)
    st.session_state["messages"].append({"role": "user", "content": q})

    with st.spinner("RealView가 열심히 정보를 분석 중..."):
        hits = db.similarity_search(q, k=5)
        context = "\n\n".join(d.page_content for d in hits)
        answer = chain.invoke({'context' : context, "q": q})

    with st.chat_message("assistant"):
        st.markdown(answer)
    st.session_state["messages"].append({"role": "assistant", "content": answer})

Writing app.py


In [6]:
import os
import time
import socket
from pyngrok import ngrok
from pyngrok.exception import PyngrokNgrokHTTPError, PyngrokNgrokURLError

# 이전 ngrok 터널 종료 (안전하게 시작하기 위해)
try:
    print("기존 ngrok 터널을 종료 시도합니다...")
    ngrok.kill()
    print("기존 ngrok 터널 종료 완료.")
except Exception as e:
    print(f"기존 ngrok 터널 종료 중 오류 발생 (이미 종료되었을 수 있음 또는 실행 중인 ngrok 없음): {e}")

# Streamlit 앱 백그라운드에서 실행
print("Streamlit 앱을 백그라운드에서 실행합니다...")
os.system("streamlit run app.py --server.address 0.0.0.0 --server.port 8501 &>/dev/null &")

# Streamlit 앱이 8501 포트에서 응답할 때까지 기다리는 함수
def wait_for_streamlit_app(port, timeout=90):
    start_time = time.time()
    while True:
        if time.time() - start_time > timeout:
            print(f"❌ {timeout}초 동안 Streamlit 앱이 {port}번 포트에서 시작되지 않았습니다.")
            return False
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.settimeout(1)
                s.connect(("127.0.0.1", port))
            print(f"✅ Streamlit 앱이 {port}번 포트에서 응답을 시작했습니다!")
            return True
        except (socket.error, ConnectionRefusedError):
            print(f"⏳ Streamlit 앱 대기 중... ({int(time.time() - start_time)}초 경과 / 최대 {timeout}초)")
            time.sleep(2)

# Streamlit 앱이 완전히 로드될 때까지 기다립니다.
if not wait_for_streamlit_app(8501, timeout=90):
    print("❌ Streamlit 앱 시작에 실패하여 ngrok 연결을 시도하지 않습니다.")
else:
    # ngrok 터널 열기
    print("ngrok 터널을 연결합니다...")
    try:
        public_url = ngrok.connect(addr="http://127.0.0.1:8501", bind_tls=True)
        print("🌐 Streamlit 앱이 실행되었습니다! 아래 링크를 클릭하세요:")
        print(public_url)
    except (PyngrokNgrokHTTPError, PyngrokNgrokURLError) as e:
        print(f"❌ ngrok 터널 연결 중 오류 발생: {type(e).__name__} - {e}")
        print("pyngrok 또는 ngrok 서비스 상태를 확인해주세요.")
        print("1. 'ngrok config add-authtoken YOUR_AUTH_TOKEN' 명령어로 인증 토큰을 다시 설정해보세요. (반드시 큰따옴표 안에 토큰을 넣어주세요!)")
        print("2. ngrok 유료 플랜이 아닌 경우, 연결 요청이 너무 많아서 일시적으로 차단되었을 수도 있습니다. 잠시 후 다시 시도해 보세요.")
        print("3. 코랩 세션 초기화 후 모든 셀을 순서대로 다시 실행해 보세요.")
    except Exception as e:
        print(f"❌ 예상치 못한 오류 발생: {type(e).__name__} - {e}")

기존 ngrok 터널을 종료 시도합니다...
기존 ngrok 터널 종료 완료.
Streamlit 앱을 백그라운드에서 실행합니다...
⏳ Streamlit 앱 대기 중... (0초 경과 / 최대 90초)
⏳ Streamlit 앱 대기 중... (2초 경과 / 최대 90초)
⏳ Streamlit 앱 대기 중... (4초 경과 / 최대 90초)
⏳ Streamlit 앱 대기 중... (6초 경과 / 최대 90초)
✅ Streamlit 앱이 8501번 포트에서 응답을 시작했습니다!
ngrok 터널을 연결합니다...
🌐 Streamlit 앱이 실행되었습니다! 아래 링크를 클릭하세요:
NgrokTunnel: "https://363946e5a2ae.ngrok-free.app" -> "http://127.0.0.1:8501"
